In [1]:
import yfinance as yf
import talib
import numpy as np
from package import indicator as ind
import pandas as pd
from pandas_datareader import data as pdr
import random
from sklearn.decomposition import PCA
import sklearn.metrics
from genetic_selection import GeneticSelectionCV
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import datetime as datetime
import warnings
from package import euklid_regressor as eur
warnings.filterwarnings("ignore")

random.seed(42)
stock = "OIL"

In [2]:
#import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)
# download dataframe
start = datetime.datetime(2000,8,23)
end = datetime.datetime(2022,4,1)
data = pdr.get_data_yahoo("CL=F", start=start, end=end)


[*********************100%***********************]  1 of 1 completed


In [3]:
data['vwap'] = eur.vwap(data['Close'], data['Volume'])
data['SO'] = eur.Stochastic_Oscillator(data['Close'])
data["ROC_1"] =  talib.ROC(data['Close'], timeperiod=1)
data["ROC_2"] =  talib.ROC(data['Close'], timeperiod=2)
data["ROC_3"] =  talib.ROC(data['Close'], timeperiod=3)
data["ROC_4"] =  talib.ROC(data['Close'], timeperiod=4)
data["ROC_5"] =  talib.ROC(data['Close'], timeperiod=5)
data["ROC_6"] =  talib.ROC(data['Close'], timeperiod=6)
data["ROC_7"] =  talib.ROC(data['Close'], timeperiod=7)
data["ROC_30"] =  talib.ROC(data['Close'], timeperiod=30)
data["ROC_60"] =  talib.ROC(data['Close'], timeperiod=60)
data['Momentum'] = ind.momentum(data)
data['Return'] = data.Close.pct_change()
data['1 Day ROI']=ind.ROI(data,1)
data['2 Day ROI']=ind.ROI(data,2)
data['3 Day ROI']=ind.ROI(data,3)
data['4 Day ROI']=ind.ROI(data,4)
data['5 Day ROI']=ind.ROI(data,5)
data['6 Day ROI']=ind.ROI(data,6)
data['7 Day ROI']=ind.ROI(data,7)
data['10 Day ROI']=ind.ROI(data,10)
data['20 Day ROI']=ind.ROI(data,20)
data['30 Day ROI']=ind.ROI(data,30)
data['RSI_1'] = eur.rsi(data['Close'],1)
data['RSI_2'] = eur.rsi(data['Close'],2)
data['RSI_3'] = eur.rsi(data['Close'],3)
data['RSI_4'] = eur.rsi(data['Close'],4)
data['RSI_5'] = eur.rsi(data['Close'],5)
data['RSI_6'] = eur.rsi(data['Close'],6)
data['RSI_7'] = eur.rsi(data['Close'],7)
data['RSI_14'] = eur.rsi(data['Close'],14)
data['RSI_30'] = eur.rsi(data['Close'],30)
data['RSI_60'] = eur.rsi(data['Close'],60)
data['EMA_12'] = ind.EMA(data, 12)
data['EMA_26'] = ind.EMA(data, 26)
data['Up down'] = ind.clas(data, .01)
eur.macd(data,data['Close'])
#Add SRSI to our dataframe
data['SRSI_10'] = ind.SRSI(data,10)
data['SRSI_14'] = ind.SRSI(data,14)
data['SRSI_30'] = ind.SRSI(data,30)
#data['Williams_1'] = ind.Williams(data,1)
#data['Williams_2'] = ind.Williams(data,2)
#data['Williams_3'] = ind.Williams(data,3)
data['Williams_14'] = ind.Williams(data,14)
data['ATR_14'] = ind.ATR(data,14) 
data['CCI'] = ind.CCI(data,20) 
#data['SAR'] = talib.SAR(data['High'], data['Low'], acceleration=0.02, maximum=0)
#data['AROONOSC_7'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=7)
#data['AROONOSC_14'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=14)
#data['AROONOSC_21'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=21)
data["APO"] = talib.APO(data['Close'], fastperiod=12, slowperiod=26, matype=0)
differenced = (data['Close'] - data['Close'].shift(1))/(data['Close'].shift(1))[1:]
previous_differenced = differenced.shift(1)
data["Differenced"] = differenced
data["Previous_differenced"] = previous_differenced

data=data.drop(['Adj Close','Volume'], axis=1)
data = data.dropna()


# Cleaning

In [4]:
data = data.dropna()
data

,Open,High,Low,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,...,MACDs_12_26_9,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced
Date,,,,,,,,,,,,,,,,,,,,,
2000-11-16,35.250000,35.549999,34.950001,35.049999,33.651578,82.026154,-1.544942,0.573890,1.800752,2.816072,...,0.209253,0.997074,1,0,-16.901458,0.844139,140.457567,0.134359,-0.015449,0.021521
2000-11-17,34.700001,35.849998,34.599998,35.599998,33.876707,100.000000,1.569185,0.000000,2.152080,3.398194,...,0.274687,1,1,0,-6.666667,0.898253,132.432546,0.353590,0.015692,-0.015449
2000-11-20,35.500000,35.849998,35.000000,35.220001,34.027371,87.581779,-1.067408,0.485027,-1.067408,1.061701,...,0.332354,0.875924,0.933866,-3.782286,-16.799927,0.891819,115.310743,0.570385,-0.010674,0.015692
2000-11-21,35.400002,35.700001,35.000000,35.150002,34.117923,85.294206,-0.198750,-1.264036,0.285313,-1.264036,...,0.380255,0.806826,0.861631,-0.961901,-18.666585,0.866244,98.120442,0.704039,-0.001987,-0.010674
2000-11-22,34.900002,35.639999,34.669998,35.400002,34.255357,93.464147,0.711238,0.511074,-0.561789,0.998580,...,0.422646,0.862101,1,0.094982,-11.999919,0.880078,85.185671,0.821475,0.007112,-0.001987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,111.750000,114.120003,108.680000,113.900002,110.122239,65.806013,1.388646,-0.896197,1.914817,1.587584,...,4.182215,0.558515,0.438054,0.615263,-43.274852,7.429509,34.459047,2.298911,0.013886,-0.022535
2022-03-28,112.919998,112.930000,102.830002,105.959999,108.741767,38.101883,-6.971029,-5.679186,-7.804752,-5.189695,...,4.105561,0.536267,0.341117,0.549592,-62.683875,7.914908,-17.115173,1.747757,-0.069710,0.013886
2022-03-29,103.470001,107.839996,98.440002,104.239998,106.560467,46.254385,-1.623255,-8.481127,-7.210253,-9.301316,...,3.919161,0.608878,0.440235,0.527793,-53.656431,8.112919,-55.162676,0.817052,-0.016233,-0.069710


# Trying to add also PCA (Feature augmentation)

In [5]:
y = data["Up down"]
normalized = ind.normalize(data.drop(columns=["Up down", "Differenced"]))
X = normalized


In [6]:
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(X)
data["principalDf"] = principalComponents

# Save data

In [13]:
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data.to_csv("data/"+stock+".csv")

# Feature selection with GANA

In [8]:
#Remove the first 30 index which could have a value 'N'
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data=data.drop(['High','Low'], axis=1)
data

,Open,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,ROC_5,ROC_6,...,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced,principalDf
Date,,,,,,,,,,,,,,,,,,,,,
2001-02-15,29.299999,28.799999,30.117018,3.496452,-3.193280,-5.075811,-5.728314,-7.246377,-8.745248,-7.692312,...,0.43764,0.422528,0.675921,-84.848502,1.03845,-84.112747,0.192756,-0.031933,-0.019446,5.392748
2001-02-16,29.200001,29.160000,30.134888,16.083891,1.250002,-1.983194,-3.889256,-4.549916,-6.086955,-7.604562,...,0.281624,0.51653,0.684177,-74.931128,0.95999,-60.728753,0.244167,0.012500,-0.031933,3.928155
2001-02-20,29.500000,28.580000,30.093268,0.000000,-1.989026,-0.763887,-3.932773,-5.800924,-6.448443,-7.954909,...,0.294661,0.46692,0.653493,-90.909091,0.982659,-81.877617,0.177885,-0.019890,0.012500,5.125830
2001-02-21,28.900000,28.549999,30.065926,0.000000,-0.104971,-2.091909,-0.868056,-4.033616,-5.899805,-6.546645,...,0.304404,0.494305,0.704176,-93.01676,0.944971,-92.071736,0.010577,-0.001050,-0.019890,4.617870
2001-02-22,28.850000,28.820000,30.074726,8.970114,0.945711,0.839747,-1.165981,0.069446,-3.126051,-5.009889,...,0.187097,0.411966,0.696685,-85.474847,0.932308,-75.72609,-0.070577,0.009457,-0.001050,3.172954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,108.849998,114.930000,110.016440,69.399868,2.836434,2.506241,9.770777,11.604192,20.928029,19.172540,...,0.57618,0.541041,0.628693,-42.115226,7.997985,50.618855,4.370321,0.028364,-0.003211,-8.544433
2022-03-24,114.470001,112.339996,110.407998,60.362867,-2.253549,0.518964,0.196213,7.297039,9.089136,18.202857,...,0.577451,0.457531,0.609231,-49.120917,7.735587,47.536735,2.644039,-0.022535,0.028364,-5.456164
2022-03-25,111.750000,113.900002,110.122239,65.806013,1.388646,-0.896197,1.914817,1.587584,8.787015,10.603999,...,0.558515,0.438054,0.615263,-43.274852,7.429509,34.459047,2.298911,0.013886,-0.022535,-5.555525


In [9]:
y = data["Up down"]
y = y.astype('int')
#Normalize the data that we have filtered
normalized = ind.normalize(data.drop(columns=["Up down","Differenced"]))
X = normalized


In [10]:
random.seed(42)
np.random.seed(42)

estimator = DecisionTreeClassifier()
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=500,
        crossover_proba=0.5,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.5,
        mutation_independent_proba=0.15,
        tournament_size=4,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	500   	[  0.499417  15.436      0.009069]	[ 0.007153  8.682966  0.004611]	[ 0.478784  1.        0.000231]	[  0.521593  30.         0.024621]
1  	299   	[-59.500161  15.136     60.009012]	[ 772.308217    7.640255  772.268683]	[-10000.            1.            0.000481]	[     0.521593     31.        10000.      ]
2  	262   	[  0.505559  14.912      0.009384]	[ 0.007921  7.367785  0.004526]      	[ 0.482557  1.        0.000356]            	[  0.521593  30.         0.024172]         
3  	293   	[-79.4976    14.144     80.009163]	[ 890.887415    7.191054  890.841475]	[-10000.            0.            0.000404]	[     0.522347     32.        10000.      ]
4  	282   	[  0.507366  13.134      0.009078]	[ 0.009376  7.011708  0.004281]      	[ 0.478409  1.        0.000742]            	[  0.523477  30.       

In [11]:
random.seed(42)
np.random.seed(42)

estimator = linear_model.LogisticRegression(C=1e-05, penalty='none')
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=1000,
        crossover_proba=0.10,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.05,
        mutation_independent_proba=0.05,
        tournament_size=5,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	1000  	[  0.51224   15.348      0.010395]	[ 0.00667   8.652104  0.006405]	[ 0.495194  1.        0.000128]	[  0.530832  30.         0.029551]
1  	248   	[ 0.519628  9.794     0.006077]   	[ 0.004551  7.13187   0.003665]	[ 0.50217   1.        0.000128]	[  0.530832  30.         0.024432]
2  	276   	[-9.476771  8.029    10.005708]   	[ 316.086167    5.421822  316.069432]	[-10000.            0.            0.000128]	[     0.531396     27.        10000.      ]
3  	289   	[ 0.526502  7.737     0.005993]   	[ 0.003772  5.426217  0.00356 ]      	[ 0.505565  1.        0.000138]            	[  0.532905  25.         0.016278]         
4  	324   	[ 0.528305  7.8       0.006815]   	[ 0.00392   5.558417  0.004196]      	[ 0.504623  1.        0.000479]            	[  0.533659  21.         0.019477]         
5  	28

47 	318   	[  0.541845  19.13       0.012758]	[ 0.006728  0.804425  0.003207]      	[  0.507451  15.         0.000675]         	[  0.544784  24.         0.022022]         
48 	282   	[  0.542     19.096      0.012678]	[ 0.006575  0.675858  0.003088]      	[  0.512918  16.         0.000576]         	[  0.544784  24.         0.020076]         
49 	249   	[  0.542355  19.099      0.0128  ]	[ 0.006289  0.647456  0.003015]      	[  0.507828  16.         0.000823]         	[  0.544784  24.         0.021669]         
50 	274   	[  0.541922  19.168      0.012752]	[ 0.006865  0.75748   0.003101]      	[  0.501982  16.         0.000231]         	[  0.544784  25.         0.023681]         
51 	256   	[  0.542075  19.112      0.012685]	[ 0.006439  0.741253  0.003053]      	[  0.508016  15.         0.000823]         	[  0.544784  24.         0.022124]         
52 	290   	[  0.541935  19.116      0.012662]	[ 0.006794  0.697527  0.003124]      	[  0.511223  15.         0.001051]         	[  0.544784 